A notebook to make p-value images from the results of whole_bain_linear_mdl_fit

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import copy
import glob
import os
from pathlib import Path
import pickle

import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyqtgraph as pg
import tifffile

from janelia_core.stats.regression import grouped_linear_regression_boot_strap_stats
from janelia_core.dataprocessing.dataset import ROIDataset
from janelia_core.utils.data_saving import append_ts
from janelia_core.visualization.volume_visualization import make_z_plane_movie
from janelia_core.visualization.custom_color_maps import make_red_green_c_map
from janelia_core.visualization.volume_visualization import make_rgb_z_plane_movie
from janelia_core.visualization.image_generation import rgb_3d_max_project
from janelia_core.visualization.volume_visualization import visualize_rgb_max_project

from keller_zlatic_vnc.visualization import gen_coef_p_vl_cmap
from keller_zlatic_vnc.visualization import visualize_coef_p_vl_max_projs


In [3]:
%matplotlib qt

## Parameters go here

In [4]:
ps = {}

# Location of results of whole_brain_linear_mdl_fit
ps['results_folder'] = r'\\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats'
#ps['results_file'] = 'whole_brain_boot_strap_2019_10_29_10_19_57_489328.pkl'
ps['results_file'] = 'whole_brain_boot_strap_2020_01_25_15_30_35_692930.pkl'

# Specify type of images we generate
ps['gen_coef_movies'] = True
ps['gen_coef_tiffs'] = True

ps['gen_p_value_movies'] = True
ps['gen_p_value_tiffs'] = True

ps['gen_filtered_coef_movies'] = True
ps['gen_filtered_coef_tiffs'] = True

ps['gen_combined_movies'] = True
ps['gen_combined_tiffs'] = True
ps['gen_combined_projs'] = True

# Threshold p-values if we are making threshold images - we will make an image for each p-value
ps['thresholds'] = [.05, .01, .002]

# Specify percentiles we use for mapping min and max values to colors
ps['data_clim_percs'] = [0.1, 99.9]

# Specify the largest log p value that is assigned a non-black color in the combined maps
ps['log_p_vl_cmap_max_vl'] = np.log10(.05)

# Specify limits when plotting colormap of log p-values (this is for the colormap legend only, it does not affect the range
# of p-values that are shown in the generated maps)
ps['log_p_vls_cmap_range'] = (0, -3, -.01)


# Specify percentiles we use for mapping min and max values to colors for mean image
ps['mean_img_clim_percs'] = [0.1, 99.9]

# Specify where the original datasets are located - we use these for determining the position of the rois
ps['data_loc_file'] = r'A:\projects\keller_vnc\data\experiment_data_locations.xlsx'
ps['dataset_folder'] = 'extracted'
ps['dataset_base_folder'] = r'K:\\SV4'
ps['roi_group'] = 'rois_1_5_5'

# Specify where we save images
ps['save_folder'] = r'\\dm11\bishoplab\projects\keller_vnc\results\whole_brain_stats'
ps['save_str'] = 'whole_brain_stats_images'


## Load the results

In [5]:
rs_file = Path(ps['results_folder']) / ps['results_file']
with open(rs_file, 'rb') as f:
    rs = pickle.load(f)
    
one_hot_vars = rs['one_hot_vars']
before_bs_rs = rs['before_bs_rs']
after_bs_rs = rs['after_bs_rs']

## Change the name of the mean variable to baseline

In [6]:
one_hot_vars[-1] = 'baseline'

## Load a dataset

Because the rois are in the same location for each dataset, we can just look at the first dataset to find the position of the rois

In [7]:
# Read in dataset locations
def c_fcn(str):
    return str.replace("'", "")
converters = {0:c_fcn, 1:c_fcn}

data_locs = pd.read_excel(ps['data_loc_file'], header=1, usecols=[1, 2], converters=converters)

# Read in the first dataset
dataset_path = (Path(ps['dataset_base_folder']) / data_locs['Main folder'][0] / data_locs['Subfolder'][0] / 
                    Path(ps['dataset_folder']) / '*.pkl')
dataset_file = glob.glob(str(dataset_path))[0]

with open(dataset_file, 'rb') as f:
    dataset = ROIDataset.from_dict(pickle.load(f))

## Get ROI locations for first dataset

In [8]:
rois = dataset.roi_groups[ps['roi_group']]['rois']
if len(rois) != len(before_bs_rs):
    raise(RuntimeError('Number of rois in dataset does not match number of rois statistics are calculated for.'))
n_rois = len(rois)

## Calculate statistics

In [9]:
before_stats = [None]*n_rois
after_stats = [None]*n_rois
for rs_i, (before_rs, after_rs) in enumerate(zip(before_bs_rs, after_bs_rs)):
    before_stats[rs_i] = grouped_linear_regression_boot_strap_stats(before_rs[0], .01)
    after_stats[rs_i] = grouped_linear_regression_boot_strap_stats(after_rs[0], .01)
    if rs_i % 10000 == 0: 
        print('Done with ' + str(rs_i) + ' rois.')

Done with 0 rois.
Done with 10000 rois.
Done with 20000 rois.
Done with 30000 rois.
Done with 40000 rois.
Done with 50000 rois.
Done with 60000 rois.
Done with 70000 rois.
Done with 80000 rois.
Done with 90000 rois.
Done with 100000 rois.
Done with 110000 rois.
Done with 120000 rois.
Done with 130000 rois.
Done with 140000 rois.
Done with 150000 rois.
Done with 160000 rois.
Done with 170000 rois.
Done with 180000 rois.
Done with 190000 rois.
Done with 200000 rois.
Done with 210000 rois.
Done with 220000 rois.
Done with 230000 rois.
Done with 240000 rois.
Done with 250000 rois.
Done with 260000 rois.
Done with 270000 rois.
Done with 280000 rois.
Done with 290000 rois.
Done with 300000 rois.
Done with 310000 rois.
Done with 320000 rois.
Done with 330000 rois.
Done with 340000 rois.
Done with 350000 rois.
Done with 360000 rois.
Done with 370000 rois.
Done with 380000 rois.


## Load mean image

In [10]:
mn_img = dataset.stats['mean']

## Generate and save images

In [11]:
# Make folders to save results into
image_folder = append_ts(ps['save_str'])
save_folder_path= Path(ps['save_folder']) / image_folder
before_stim_folder = save_folder_path / 'before_pert_mdls'
after_stim_folder = save_folder_path / 'after_pert_mdls'

os.makedirs(save_folder_path)
os.makedirs(before_stim_folder)
os.makedirs(after_stim_folder)

In [12]:
# Save the mean image
mn_image_path = save_folder_path / 'mean.tiff'

imageio.mimwrite(mn_image_path, mn_img)

mn_img_min_c_lim = np.percentile(mn_img,  ps['mean_img_clim_percs'][0])
mn_img_max_c_lim = np.percentile(mn_img, ps['mean_img_clim_percs'][1])

make_z_plane_movie(volume=mn_img, save_path= str(save_folder_path /  'mean.mp4'), 
                   cmap='gray', clim=(mn_img_min_c_lim, mn_img_max_c_lim),
                   title = 'Mean Image', cbar_label='$F$')

In [13]:
im_shape = mn_img.shape

n_vars = len(one_hot_vars)
min_log_p_vl = np.log10(1/rs['ps']['n_bs_smps'])

coef_cmap = make_red_green_c_map(1024)

for v_i in range(n_vars):
    var_name = one_hot_vars[v_i]
    before_stim_base_name = var_name + '_before_pert_mdl'
    after_stim_base_name = var_name + '_after_pert_mdl'
    before_stim_title = var_name + ', before_pert_mdl'
    after_stim_title = var_name + ', after_pert_mdl'
    
    var_before_stim_folder = before_stim_folder / var_name
    var_after_stim_folder = after_stim_folder / var_name
    os.makedirs(var_before_stim_folder)
    os.makedirs(var_after_stim_folder)
    
    
    before_stim_coefs = np.zeros(im_shape, dtype=np.float32) 
    after_stim_coefs = np.zeros(im_shape, dtype=np.float32) 
    
    before_stim_p_vls = np.zeros(im_shape, dtype=np.float32)
    after_stim_p_vls = np.zeros(im_shape, dtype=np.float32) 
    
    for r_i in range(n_rois):
        cur_voxel_inds = rois[r_i].voxel_inds
        
        before_stim_coefs[cur_voxel_inds] = before_bs_rs[r_i][1][v_i]
        after_stim_coefs[cur_voxel_inds] = after_bs_rs[r_i][1][v_i]
        
        before_stim_p_vls[cur_voxel_inds] = before_stats[r_i]['non_zero_p'][v_i]
        after_stim_p_vls[cur_voxel_inds] = after_stats[r_i]['non_zero_p'][v_i]
        
    if ps['gen_coef_movies'] or ps['gen_coef_tiffs']:
        before_stim_file_name = before_stim_base_name + '_coefs'
        after_stim_file_name = after_stim_base_name + '_coefs'
        
        if ps['gen_coef_tiffs']:
            tifffile.imwrite(var_before_stim_folder / (before_stim_file_name + '.tiff'), before_stim_coefs, compress=6)
            tifffile.imwrite(var_after_stim_folder / (after_stim_file_name + '.tiff'), after_stim_coefs, compress=6)
    
        if ps['gen_coef_movies']:
            make_z_plane_movie(volume=before_stim_coefs, save_path=str(var_before_stim_folder / (before_stim_file_name + '.mp4')), 
                               cmap=coef_cmap, clim=None, clim_percs = ps['data_clim_percs'],
                               title = before_stim_title, cbar_label='${\Delta F}/{F}$')
        
            make_z_plane_movie(volume=after_stim_coefs, save_path=str(var_after_stim_folder / (after_stim_file_name + '.mp4')), 
                               cmap=coef_cmap, clim=None, clim_percs = ps['data_clim_percs'],
                               title = after_stim_title, cbar_label='${\Delta F}/{F}$')
        
    if ps['gen_p_value_movies'] or ps['gen_p_value_tiffs']:
        before_stim_file_name = before_stim_base_name + '_p_vls'
        after_stim_file_name = after_stim_base_name + '_p_vls'
        
        if ps['gen_p_value_tiffs']:
            tifffile.imwrite(var_before_stim_folder / (before_stim_file_name + '.tiff'), before_stim_p_vls, compress=6)
            tifffile.imwrite(var_after_stim_folder / (after_stim_file_name + '.tiff'), after_stim_p_vls, compress=6)
             
        if ps['gen_p_value_movies']:
            before_stim_log_p_vls = np.log10(before_stim_p_vls)
            before_stim_log_p_vls[before_stim_p_vls == 0] = 0
            after_stim_log_p_vls = np.log10(after_stim_p_vls)
            after_stim_log_p_vls[after_stim_p_vls == 0] = 0
        
            make_z_plane_movie(volume=before_stim_log_p_vls, save_path=str(var_before_stim_folder / (before_stim_file_name + '.mp4')), 
                               cmap='magma_r', clim=(min_log_p_vl, 0), title = before_stim_title, cbar_label='$\log_{10}(p)$')
            make_z_plane_movie(volume=after_stim_log_p_vls, save_path=str(var_after_stim_folder / (after_stim_file_name + '.mp4')), 
                               cmap='magma_r', clim=(min_log_p_vl, 0), title = after_stim_title, cbar_label='$\log_{10}(p)$')
    
    if ps['gen_filtered_coef_movies'] or ps['gen_filtered_coef_tiffs']:
        for th in ps['thresholds']:
            before_stim_file_name = before_stim_base_name + '_coefs_p_th_' + str(th)
            after_stim_file_name = after_stim_base_name + '_coefs_p_th_' + str(th)
            
            before_stim_coefs_th = copy.deepcopy(before_stim_coefs)
            after_stim_coefs_th = copy.deepcopy(after_stim_coefs)
            
            before_stim_coefs_th[before_stim_p_vls > th] = 0
            after_stim_coefs_th[after_stim_p_vls > th] = 0
            
            if ps['gen_filtered_coef_tiffs']:
                tifffile.imwrite(var_before_stim_folder / (before_stim_file_name + '.tiff'), before_stim_coefs_th, compress=6)
                tifffile.imwrite(var_after_stim_folder / (after_stim_file_name + '.tiff'), after_stim_coefs_th, compress=6)
               
            if ps['gen_filtered_coef_movies']:
                make_z_plane_movie(volume=before_stim_coefs_th, save_path=str(var_before_stim_folder / (before_stim_file_name + '.mp4')), 
                                   cmap=coef_cmap, clim=None, clim_percs = ps['data_clim_percs'],
                                   title = before_stim_title + '$, p \leq$' + str(th), cbar_label='${\Delta F}/{F}$')
            
                make_z_plane_movie(volume=after_stim_coefs_th, save_path=str(var_after_stim_folder / (after_stim_file_name + '.mp4')), 
                                   cmap=coef_cmap, clim=None, clim_percs = ps['data_clim_percs'],
                                   title = after_stim_title + '$, p \leq$' + str(th), cbar_label='${\Delta F}/{F}$')
            
    if ps['gen_combined_movies'] or ps['gen_combined_tiffs'] or ps['gen_combined_projs']:
        before_stim_file_name = before_stim_base_name + '_combined'
        after_stim_file_name = after_stim_base_name + '_combined' 
        
        before_stim_log_p_vls = np.log10(before_stim_p_vls)
        before_stim_log_p_vls[before_stim_p_vls == 0] = 0
        after_stim_log_p_vls = np.log10(after_stim_p_vls)
        after_stim_log_p_vls[after_stim_p_vls == 0] = 0
        
        # Generate color maps for before and after variables
        before_cmap = gen_coef_p_vl_cmap(coef_vls = before_stim_coefs, p_vls=before_stim_log_p_vls, 
                                         clim_percs = ps['data_clim_percs'], 
                                         max_p_vl=ps['log_p_vl_cmap_max_vl'])
        after_cmap =  gen_coef_p_vl_cmap(coef_vls = after_stim_coefs, p_vls=after_stim_log_p_vls, 
                                         clim_percs = ps['data_clim_percs'],
                                         max_p_vl=ps['log_p_vl_cmap_max_vl'])
        
        # Make RGB volumes 
        before_combined_vol = before_cmap[before_stim_coefs, before_stim_log_p_vls]
        after_combined_vol = after_cmap[after_stim_coefs, after_stim_log_p_vls]
        
        before_combined_vol_uint8 = (before_combined_vol*255).astype(np.uint8)
        after_combined_vol_uint8 = (after_combined_vol*255).astype(np.uint8)
        
        n_z_planes = before_stim_coefs.shape[0]
        before_combined_planes = [np.squeeze(before_combined_vol[z, :,:,:]) for z in range(n_z_planes)]
        after_combined_planes = [np.squeeze(after_combined_vol[z, :,:,:]) for z in range(n_z_planes)]
        
        # Save tiff stacks of RGB volumes
        if ps['gen_combined_tiffs']:
            tifffile.imwrite(var_before_stim_folder / (before_stim_file_name + '.tiff'), before_combined_vol_uint8, compress=6)
            tifffile.imwrite(var_after_stim_folder / (after_stim_file_name + '.tiff'), after_combined_vol_uint8, compress=6)
   
            # Save colormaps for combined tiffs
            before_combined_cmap_file = var_before_stim_folder / (before_stim_file_name + '_cmap.pkl')
            with open(before_combined_cmap_file, 'wb') as f:
                pickle.dump(before_cmap.to_dict(), f)
            after_combined_cmap_file = var_after_stim_folder / (after_stim_file_name + '_cmap.pkl')
            with open(after_combined_cmap_file, 'wb') as f:
                pickle.dump(after_cmap.to_dict(), f)

        # Make videos of RGB volumes
        if ps['gen_combined_movies']:
            make_rgb_z_plane_movie(z_imgs=before_combined_planes, 
                                   save_path=str(var_before_stim_folder / (before_stim_file_name + '.mp4')),
                                   cmap=before_cmap, 
                                   title=before_stim_title,
                                   cmap_param_vls=(None, np.arange(*ps['log_p_vls_cmap_range'])),
                                   cmap_param_strs = ['coef vl ($\Delta F / F$)', '$\log(p)$'],
                                   one_index_z_plane=True)
                               
            make_rgb_z_plane_movie(z_imgs=after_combined_planes, 
                                   save_path=str(var_after_stim_folder / (after_stim_file_name + '.mp4')),
                                   cmap=after_cmap, 
                                   title=after_stim_title,
                                   cmap_param_vls=(None, np.arange(*ps['log_p_vls_cmap_range'])),
                                   cmap_param_strs = ['coef vl ($\Delta F / F$)', '$\log(p)$'], 
                                   one_index_z_plane=True)
            
        if ps['gen_combined_projs']:
            
            visualize_coef_p_vl_max_projs(vol=np.moveaxis(before_combined_vol, 0, 2), dim_m=np.asarray([1, 1, 5]), 
                                          cmap=before_cmap,
                                          cmap_coef_range=None, cmap_p_vl_range=ps['log_p_vls_cmap_range'],
                                          title=before_stim_title)
            plt.savefig(var_before_stim_folder / (before_stim_file_name + '.png'), facecolor=(0,0,0))
            plt.close()
            
            visualize_coef_p_vl_max_projs(vol=np.moveaxis(after_combined_vol, 0, 2), dim_m=np.asarray([1, 1, 5]), 
                                          cmap=after_cmap,
                                          cmap_coef_range=None, cmap_p_vl_range=ps['log_p_vls_cmap_range'],
                                          title=after_stim_title)
            plt.savefig(var_after_stim_folder / (after_stim_file_name + '.png'), facecolor=(0,0,0))
            plt.close()
    
    print('Done with making images for variable: ' + var_name)
        
        
    

C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:100: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:102: RuntimeWarning: divide by zero encountered in log10


Done with making images for variable: beh_before_Q


C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:100: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:102: RuntimeWarning: divide by zero encountered in log10


Done with making images for variable: beh_before_F


C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:100: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:102: RuntimeWarning: divide by zero encountered in log10


...

C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:100: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:102: RuntimeWarning: divide by zero encountered in log10


Done with making images for variable: beh_interact_BB


C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:62: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:64: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:100: RuntimeWarning: divide by zero encountered in log10
C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\ipykernel_launcher.py:102: RuntimeWarning: divide by zero encountered in log10


Done with making images for variable: baseline


Traceback (most recent call last):
  File "C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\matplotlib-3.1.1-py3.7-win-amd64.egg\matplotlib\cbook\__init__.py", line 216, in process
    func(*args, **kwargs)
  File "C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\matplotlib-3.1.1-py3.7-win-amd64.egg\matplotlib\animation.py", line 957, in _start
    self.event_source.add_callback(self._step)
AttributeError: 'NoneType' object has no attribute 'add_callback'
Traceback (most recent call last):
  File "C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\matplotlib-3.1.1-py3.7-win-amd64.egg\matplotlib\cbook\__init__.py", line 216, in process
    func(*args, **kwargs)
  File "C:\Users\bishopw\AppData\Local\Continuum\anaconda3\envs\keller_zlatic_vnc\lib\site-packages\matplotlib-3.1.1-py3.7-win-amd64.egg\matplotlib\animation.py", line 957, in _start
    self.event_source

In [ ]:
vars(before_cmap)

In [ ]:
before_combined_vol_uint8 = (before_combined_vol*255).astype(np.uint8)

In [ ]:
np.max(before_combined_vol_uint8)

In [ ]:
ps['log_p_vls_cmap_range']

In [ ]:
visualize_coef_p_vl_max_projs(vol=np.moveaxis(before_combined_vol, 0, 2), cmap=before_cmap,
                                        cmap_coef_range=None, cmap_p_vl_range=(0, -3, -.01))
plt.savefig(var_after_stim_folder / (after_stim_file_name + '.png'), facecolor=(0,0,0))

In [ ]:
## Save debug stuff

In [ ]:
save_file = r'C:\Users\bishopw\Desktop\debug.pkl'
rs = {'before_stim_p_vls': before_stim_p_vls, 
      'after_stim_p_vls': after_stim_p_vls,
      'before_stim_ceofs': before_stim_coefs,
      'after_stim_coesfs': after_stim_coefs}

with open(save_file, 'wb') as f:
    pickle.dump(rs, f)

## Load debug stuff

In [ ]:
debug_file = r'C:\Users\bishopw\Desktop\debug.pkl'
with open(debug_file, 'rb') as f:
    rs = pickle.load(f)

In [ ]:
before_stim_p_vls = rs['before_stim_p_vls']
before_stim_coefs = rs['before_stim_ceofs']
after_stim_p_vls = rs['after_stim_p_vls']
after_stim_coefs = rs['after_stim_coesfs']

In [ ]:
before_stim_log_p_vls = np.log10(before_stim_p_vls)
before_stim_log_p_vls[before_stim_p_vls == 0] = 0
after_stim_log_p_vls = np.log10(after_stim_p_vls)
after_stim_log_p_vls[after_stim_p_vls == 0] = 0

In [ ]:
# Generate color maps for before and after variables
before_cmap = gen_coef_p_vl_cmap(coef_vls = before_stim_coefs, p_vls=before_stim_log_p_vls, max_p_vl=-2)
after_cmap =  gen_coef_p_vl_cmap(coef_vls = after_stim_coefs, p_vls=after_stim_log_p_vls, max_p_vl=-2)

before_combined_vol = before_cmap[before_stim_coefs, before_stim_log_p_vls]
after_combined_vol = after_cmap[after_stim_coefs, after_stim_log_p_vls]

n_z_planes = before_stim_coefs.shape[0]
before_combined_planes = [np.squeeze(before_combined_vol[z, :,:,:]) for z in range(n_z_planes)]
after_combined_planes = [np.squeeze(after_combined_vol[z, :,:,:]) for z in range(n_z_planes)]

In [ ]:
cmap_im = visualize_coef_p_vl_max_projs(vol=np.moveaxis(before_combined_vol, 0, 2), dim_m = np.asarray([1, 1, 5]), cmap=before_cmap,
                                        cmap_coef_range=None, cmap_p_vl_range=(0, -3, -.01))

In [ ]:
make_rgb_z_plane_movie(z_imgs=before_combined_planes, 
                                   save_path=r'C:\Users\bishopw\Desktop\test1.mp4',
                                   cmap=before_cmap, 
                                   title='test_title',
                                   cmap_param_vls=(None, np.arange(*ps['log_p_vls_cmap_range'])),
                                   cmap_param_strs = ['coef vl ($\Delta F / F$)', '$\log(p)$'],
                                   one_index_z_plane=True)